# Усечение

Оригинал статьи с кодом можно найти [тут](https://docs.pytorch.org/tutorials/intermediate/pruning_tutorial.html).

Этот пример лучше запускать либо на компьютере с GPU от NVidia, либо в [Google Colab](https://colab.research.google.com/).

In [ ]:
!pip3 install torch

In [ ]:
import torch

print(f"Torch version {torch.__version__}")

# Let's make sure GPU is available!
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F

Создадим модель архитектуры типа LeNet, предложенной Яном Лекуном еще в 1998 году.

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square conv kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5x5 image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, int(x.nelement() / x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = LeNet().to(device=device)

Посмотрим внимательно на первый сверточный слой.

In [ ]:
module = model.conv1
print(list(module.named_parameters()))
print(list(module.named_buffers()))

Прореживаем произвольным образом 30% весов в первом слое.

In [ ]:
prune.random_unstructured(module, name="weight", amount=0.3)

В модели появляется дополнительный параметр __weight_orig__.

In [ ]:
print(list(module.named_parameters()))

А само прореживание запоминается как маска.

In [ ]:
print(list(module.named_buffers()))

Маска объединяется с исходным атрибутом весов.

In [ ]:
print(module.weight)

Перед каждым прямым проходом выполняется усечение с использованием __forward_pre_hooks__ из PyTorch. Для каждого параметра, который используется для усечения модуля, используется свой __forward_pre_hook__. В нашем случае, поскольку мы пока усекаем только исходный параметр с именем weight, будет присутствовать только один хук.

In [ ]:
print(module._forward_pre_hooks)

Теперь усечем смещение с использованием нормы L1.

In [ ]:
prune.l1_unstructured(module, name="bias", amount=3)

Видно, что теперь у нас два параметра.

In [ ]:
print(list(module.named_parameters()))

In [ ]:
print(list(module.named_buffers()))

In [ ]:
print(module.bias)

И два хука.

In [ ]:
print(module._forward_pre_hooks)

## Итеративное усечение

Один и тот же параметр в модуле может быть усечен несколько раз, при этом эффект от различных вызовов усечения будет равен комбинации различных масок, применяемых последовательно. Комбинация новой маски со старой маской обрабатывается методом __compute_mask__ объекта __PruningContainer__.

Мы дополнительно усекаем module.weight, используя структурированное усечение вдоль нулевой оси тензора, на основе нормы L2.

In [ ]:
prune.ln_structured(module, name="weight", amount=0.5, n=2, dim=0)

# As we can verify, this will zero out all the connections corresponding to
# 50% (3 out of 6) of the channels, while preserving the action of the
# previous mask.
print(module.weight)

In [ ]:
for hook in module._forward_pre_hooks.values():
    if hook._tensor_name == "weight":  # select out the correct hook
        break

print(list(hook))  # pruning history in the container

## Сериализация усеченной модели

Все соответствующие тензоры, включая буферы масок и исходные параметры, использованные для вычисления усеченных тензоров, хранятся в словаре состояния модели (state_dict) и, следовательно, могут быть легко сериализованы и сохранены при необходимости.

In [ ]:
print(model.state_dict().keys())

Теперь зафиксируем в модели все выполненные операции усечения. Этот процесс необратим.

In [ ]:
print(list(module.named_parameters()))

In [ ]:
print(list(module.named_buffers()))

In [ ]:
print(module.weight)

In [ ]:
prune.remove(module, 'weight')
print(list(module.named_parameters()))

In [ ]:
print(list(module.named_buffers()))

Указав желаемый метод прореживания и параметры, мы можем легко усечь несколько тензоров в сети, например, в соответствии с их типом.

In [ ]:
new_model = LeNet()
for name, module in new_model.named_modules():
    # prune 20% of connections in all 2D-conv layers
    if isinstance(module, torch.nn.Conv2d):
        prune.l1_unstructured(module, name='weight', amount=0.2)
    # prune 40% of connections in all linear layers
    elif isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name='weight', amount=0.4)

print(dict(new_model.named_buffers()).keys())  # to verify that all masks exist

## Глобальное усечение

Распространенный и, возможно, более эффективный метод заключается в усечении всей модели сразу, удаляя, например, 20% самых слабых связей по всей модели, вместо удаления 20% самых слабых связей в каждом слое. Это, как правило, приводит к разным процентам усечения для каждого слоя.

In [ ]:
model = LeNet()

parameters_to_prune = (
    (model.conv1, 'weight'),
    (model.conv2, 'weight'),
    (model.fc1, 'weight'),
    (model.fc2, 'weight'),
    (model.fc3, 'weight'),
)

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.2,
)

Посмотрим какая разреженность возникает в каждом усеченном параметре, при том, что глобальная разреженность составит приблизительно 20%.

In [ ]:
print(
    "Sparsity in conv1.weight: {:.2f}%".format(
        100. * float(torch.sum(model.conv1.weight == 0))
        / float(model.conv1.weight.nelement())
    )
)
print(
    "Sparsity in conv2.weight: {:.2f}%".format(
        100. * float(torch.sum(model.conv2.weight == 0))
        / float(model.conv2.weight.nelement())
    )
)
print(
    "Sparsity in fc1.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc1.weight == 0))
        / float(model.fc1.weight.nelement())
    )
)
print(
    "Sparsity in fc2.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc2.weight == 0))
        / float(model.fc2.weight.nelement())
    )
)
print(
    "Sparsity in fc3.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc3.weight == 0))
        / float(model.fc3.weight.nelement())
    )
)
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.weight == 0)
            + torch.sum(model.conv2.weight == 0)
            + torch.sum(model.fc1.weight == 0)
            + torch.sum(model.fc2.weight == 0)
            + torch.sum(model.fc3.weight == 0)
        )
        / float(
            model.conv1.weight.nelement()
            + model.conv2.weight.nelement()
            + model.fc1.weight.nelement()
            + model.fc2.weight.nelement()
            + model.fc3.weight.nelement()
        )
    )
)

## Пользовательские функции усечения

Чтобы реализовать собственную функцию усечения, создадим подкласс базового класса __BasePruningMethod__, наподобие остальные методов.

Переопределяем __\_\_init\_\___ (конструктор) и __compute_mask__ (логика вычисления маски усечения для заданного тензора). И указываем тип усечения, который реализует новая логика (глобальный, структурированный и неструктурированный).

In [ ]:
class FooBarPruningMethod(prune.BasePruningMethod):
    """Prune every other entry in a tensor
    """
    PRUNING_TYPE = 'unstructured'

    def compute_mask(self, t, default_mask):
        mask = default_mask.clone()
        mask.view(-1)[::2] = 0
        return mask

In [ ]:
def foobar_unstructured(module, name):
    """Prunes tensor corresponding to parameter called `name` in `module`
    by removing every other entry in the tensors.
    Modifies module in place (and also return the modified module)
    by:
    1) adding a named buffer called `name+'_mask'` corresponding to the
    binary mask applied to the parameter `name` by the pruning method.
    The parameter `name` is replaced by its pruned version, while the
    original (unpruned) parameter is stored in a new parameter named
    `name+'_orig'`.

    Args:
        module (nn.Module): module containing the tensor to prune
        name (string): parameter name within `module` on which pruning
                will act.

    Returns:
        module (nn.Module): modified (i.e. pruned) version of the input
            module

    Examples:
        >>> m = nn.Linear(3, 4)
        >>> foobar_unstructured(m, name='bias')
    """
    FooBarPruningMethod.apply(module, name)
    return module

In [ ]:
model = LeNet()
foobar_unstructured(model.fc3, name='bias')

print(model.fc3.bias_mask)